In [1]:
%load_ext autoreload

from pathlib import Path

from tqdm import tqdm, trange

import numpy as np

from scipy import ndimage as ndi

import skimage
import skimage.feature
import skimage.filters
import skimage.morphology
import skimage.segmentation as seg
import skimage.measure

import cv2 as cv

from bokeh.plotting import output_notebook, figure, show
from bokeh.layouts import column, row, layout, gridplot
from bokeh.models import ColumnDataSource, Slider

import fish

output_notebook()

Loading BokehJS ...

In [2]:
THIS_DIR = Path.cwd()
ROOT_DIR = THIS_DIR.parent
DATA_DIR = ROOT_DIR / 'data'

In [3]:
def show_frame(frame):
    W = frame.shape[1]
    H = frame.shape[0]

    def _(doc):
        p = figure(match_aspect = True)
        p.x_range.range_padding = p.y_range.range_padding = 0

        source = ColumnDataSource(data=dict(image=[frame[:, :]]))

        p.image(image='image', source = source, x = 0, y = 0, dw = W, dh = H, palette="Greys256")

        doc.add_root(p)
        
    return show(_)

def show_frames(frames):
    N = frames.shape[0]
    W = frames.shape[2]
    H = frames.shape[1]

    def _(doc):
        p = figure(match_aspect = True)
        p.x_range.range_padding = p.y_range.range_padding = 0

        source = ColumnDataSource(data=dict(image=[frames[0, :, :]]))

        slider = Slider(start=0, end=(N-1), value=0, step=1, title="Frame", callback_policy='throttle', callback_throttle=100)
        
        def update(attr, old, new):
            source.data = dict(image=[frames[new, :, :]])
        
        slider.on_change('value_throttled', update)

        p.image(image='image', source = source, x = 0, y = 0, dw = W, dh = H, palette="Greys256")

        doc.add_root(column(p, slider))
        
    return show(_)

In [4]:
movie = fish.read(DATA_DIR / 'D1-1_17_medium.mp4')
movie = np.fliplr(movie)
print(movie.shape)

Reading frames from C:\Users\joshk\projects\fish\data\D1-1_17_medium.mp4: 100%|██████████████████████████████████████████████████████████████████████████████████████| 1816/1816 [00:08<00:00, 210.26it/s]

(1816, 1024, 1280)


In [5]:
show_frames(movie)

In [6]:
backsub = cv.createBackgroundSubtractorMOG2()
backsubbed_movie = np.empty_like(movie)
for frame_idx, frame in enumerate(tqdm(movie)):
    backsubbed_movie[frame_idx] = backsub.apply(frame)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1816/1816 [00:14<00:00, 126.69it/s]


In [7]:
show_frames(backsubbed_movie)

In [8]:
show_frame(backsub.getBackgroundImage())

In [9]:
avg = np.mean(movie, axis=0)
show_frame(avg)

In [10]:
shuffle_backsub = cv.createBackgroundSubtractorMOG2(detectShadows=False)

shuffled = movie.copy()

for _ in trange(1):
    np.random.shuffle(shuffled)
    for frame in shuffled:
        shuffle_backsub.apply(frame)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:18<00:00, 18.17s/it]


In [11]:
show_frame(shuffle_backsub.getBackgroundImage())

In [12]:
backsubbed_movie = np.empty_like(movie)
for frame_idx, frame in enumerate(tqdm(movie)):
    backsubbed_movie[frame_idx] = shuffle_backsub.apply(frame, learningRate = 0)  # don't update the background model, which we've pre-trained

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1816/1816 [00:28<00:00, 64.32it/s]


In [13]:
show_frames(backsubbed_movie)

In [14]:
def canny(frames):
    N = frames.shape[0]
    W = frames.shape[2]
    H = frames.shape[1]

    def _(doc):
        orig = figure(match_aspect = True)
        cann = figure(match_aspect = True)
        orig.x_range.range_padding = orig.y_range.range_padding = 0
        cann.x_range = orig.x_range
        cann.y_range = orig.y_range
        
        frame_opts = {
            'index': 0
        }
        canny_opts = {
            'sigma': 1,
            'low_threshold': 10,
            'high_threshold': 200,
        }

        orig_source = ColumnDataSource(data=dict(image=[frames[frame_opts['index'], :, :]]))
        cann_source = ColumnDataSource(data=dict(image=[skimage.feature.canny(frames[frame_opts['index'], :, :], **canny_opts)]))

        frame_slider = Slider(start=0, end=(N-1), value=frame_opts['index'], step=1, title="Frame", callback_policy='mouseup', callback_throttle=100)
        sigma_slider = Slider(start=0, end=3, value=canny_opts['sigma'], step=.01, title="Sigma", callback_policy='mouseup', callback_throttle=100)
        low_slider = Slider(start=0, end=255, value=canny_opts['low_threshold'], step=.01, title="Low Threshold", callback_policy='mouseup', callback_throttle=100)
        high_slider = Slider(start=0, end=255, value=canny_opts['high_threshold'], step=.01, title="High Threshold", callback_policy='mouseup', callback_throttle=100)
        
        def updater(opt_dict, key):
            def _(attr, old, new):
                opt_dict[key] = new
                
                f = frames[frame_opts['index']]
                orig_source.data = dict(image=[f])
                
                c = skimage.feature.canny(f, **canny_opts)
                cann_source.data = dict(image=[c])
                
            return _
        
        frame_slider.on_change('value_throttled', updater(frame_opts, 'index'))
        sigma_slider.on_change('value_throttled', updater(canny_opts, 'sigma'))
        low_slider.on_change('value_throttled', updater(canny_opts, 'low_threshold'))
        high_slider.on_change('value_throttled', updater(canny_opts, 'high_threshold'))

        orig.image(image='image', source = orig_source, x = 0, y = 0, dw = W, dh = H, palette="Greys256")
        cann.image(image='image', source = cann_source, x = 0, y = 0, dw = W, dh = H, palette="Greys256")

        doc.add_root(column(row(orig, cann), frame_slider, row(sigma_slider, low_slider, high_slider)))
        
    return show(_)

In [15]:
canny(movie)

In [16]:
frame = movie[0]

markers = np.zeros_like(frame)
markers[frame < 20] = 0
markers[frame > 100] = 2

show_frame(markers)

In [17]:
elevation = skimage.filters.scharr(frame)
show_frame(elevation)

In [18]:
segmentation = skimage.morphology.watershed(elevation, markers, compactness = .0001)
show_frame(segmentation)

In [19]:
canny(backsubbed_movie)

In [20]:
c = skimage.feature.canny(backsubbed_movie[186], low_threshold=26, high_threshold=239, sigma=1.5)

In [21]:
show_frame(c)

In [22]:
contours = skimage.measure.find_contours(c, .5)

In [23]:
len(contours)

590

In [24]:
filled = ndi.binary_fill_holes(ndi.binary_closing(c))
show_frame(filled)

In [25]:
labelled, num = ndi.label(c)
show_frame(labelled)

In [69]:
def slices(frames):
    N = frames.shape[0]
    H = frames.shape[1]
    W = frames.shape[2]

    def _(doc):
        orig = figure(match_aspect = True)
        slic = figure(match_aspect = True)
        slic.y_range.start = 0
        slic.y_range.end = 260
        
        frame_opts = {
            'index': 0,
            'h': 0,
            'v': 0,
        }

        orig_source = ColumnDataSource(data=dict(image=[frames[frame_opts['index'], :, :]]))
        slic_source = ColumnDataSource(data=dict(x=np.arange(0, H, 1), y_coord=frame_opts['h'],y=frames[frame_opts['index'], :, frame_opts['h']]))

        frame_slider = Slider(start=0, end=(N-1), value=frame_opts['index'], step=1, title="Frame", callback_policy='mouseup', callback_throttle=100)
        h_slider = Slider(start=0, end=H, value=frame_opts['h'], step=1, title="h", callback_policy='mouseup', callback_throttle=100)
        
        def updater(opt_dict, key):
            def _(attr, old, new):
                opt_dict[key] = new
                
                f = frames[frame_opts['index']]
                orig_source.data = dict(image=[f])
                
                slic_source.data = dict(x = np.arange(0, H, 1), y=f[:, frame_opts['h']], y_coord=frame_opts['h'])
                
            return _
        
        frame_slider.on_change('value_throttled', updater(frame_opts, 'index'))
        h_slider.on_change('value_throttled', updater(frame_opts, 'h'))

        orig.image(image='image', source = orig_source, x = 0, y = 0, dw = W, dh = H, palette="Greys256")
        orig.hbar(y='y', source = slic_source)
        slic.line(x='x', y='y_coord', source = slic_source)

        doc.add_root(column(row(orig, slic), frame_slider, row(h_slider)))
        
    return show(_)

In [70]:
slices(backsubbed_movie)

ERROR:tornado.application:Uncaught exception GET /autoload.js?bokeh-autoload-element=6694&bokeh-absolute-url=http://localhost:60564&resources=none (::1)
HTTPServerRequest(protocol='http', host='localhost:60564', method='GET', uri='/autoload.js?bokeh-autoload-element=6694&bokeh-absolute-url=http://localhost:60564&resources=none', version='HTTP/1.1', remote_ip='::1')
Traceback (most recent call last):
  File "C:\Users\joshk\.python\envs\fish\lib\site-packages\tornado\web.py", line 1699, in _execute
    result = await result
  File "C:\Users\joshk\.python\envs\fish\lib\site-packages\tornado\gen.py", line 742, in run
    yielded = self.gen.throw(*exc_info)  # type: ignore
  File "C:\Users\joshk\.python\envs\fish\lib\site-packages\bokeh\server\views\autoload_js_handler.py", line 60, in get
    session = yield self.get_session()
  File "C:\Users\joshk\.python\envs\fish\lib\site-packages\tornado\gen.py", line 735, in run
    value = future.result()
  File "C:\Users\joshk\.python\envs\fish\lib